In [2]:
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import numpy as np

In [3]:
from collections import Counter 

In [4]:
from sklearn.metrics import make_scorer
import pickle

In [5]:
import pandas as pd
from scipy.stats import zscore
import seaborn as sn
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import ExtraTreesClassifier,RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import f1_score,precision_score,accuracy_score,recall_score,roc_auc_score

In [6]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
test_data=pd.read_csv('Test.csv')

In [8]:
test_data.head() #to display the first 5 rows of the data

,patient_id,patient_race,payer_type,patient_state,patient_zip3,Region,Division,patient_age,patient_gender,bmi,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
0,730681,NaN,COMMERCIAL,LA,713,South,West South Central,55,F,NaN,...,62.21,62.23,78.34,81.96,83.58,82.22,80.20,69.73,53.14,51.34
1,334212,Black,NaN,NC,283,South,South Atlantic,60,F,40.00,...,48.63,58.14,77.26,80.05,82.88,82.09,78.85,64.60,50.57,48.10
2,571362,NaN,COMMERCIAL,TX,794,South,West South Central,54,F,32.33,...,57.82,59.95,77.79,82.45,82.44,80.77,72.16,59.31,48.25,42.13
3,907331,NaN,COMMERCIAL,TN,373,South,East South Central,63,F,27.07,...,47.57,53.50,71.31,75.20,76.96,75.78,74.87,61.06,44.31,42.83
4,208382,Asian,NaN,WA,980,West,Pacific,62,F,NaN,...,41.02,46.25,56.92,57.88,66.16,65.21,57.52,49.53,43.75,38.33


In [9]:
with open('column_list.pkl', 'rb') as f:
    column_list = pickle.load(f)
    
with open('cat_columns.pkl', 'rb') as f:
    cat_columns = pickle.load(f)

with open('cont_columns.pkl', 'rb') as f:
    cont_columns = pickle.load(f)

In [10]:
final_column_list=['patient_id']+column_list

In [11]:
final_column_list

['patient_id',
 'patient_race',
 'payer_type',
 'patient_state',
 'Region',
 'Division',
 'breast_cancer_diagnosis_desc',
 'metastatic_cancer_diagnosis_code',
 'patient_age',
 'population',
 'density',
 'age_median',
 'age_under_10',
 'age_10_to_19',
 'age_20s',
 'age_30s',
 'age_40s',
 'age_50s',
 'age_60s',
 'age_70s',
 'age_over_80',
 'male',
 'female',
 'married',
 'divorced',
 'never_married',
 'widowed',
 'family_size',
 'family_dual_income',
 'income_household_median',
 'income_household_under_5',
 'income_household_5_to_10',
 'income_household_10_to_15',
 'income_household_15_to_20',
 'income_household_20_to_25',
 'income_household_25_to_35',
 'income_household_35_to_50',
 'income_household_50_to_75',
 'income_household_75_to_100',
 'income_household_100_to_150',
 'income_household_150_over',
 'income_household_six_figure',
 'income_individual_median',
 'home_ownership',
 'housing_units',
 'home_value',
 'rent_median',
 'rent_burden',
 'education_less_highschool',
 'education_h

In [12]:
final_test_data=test_data[final_column_list]

In [13]:
final_test_data

,patient_id,patient_race,payer_type,patient_state,Region,Division,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,patient_age,population,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
0,730681,NaN,COMMERCIAL,LA,South,West South Central,Malignant neoplasm of axillary tail of female ...,C7981,55,4639.07,...,62.21,62.23,78.34,81.96,83.58,82.22,80.20,69.73,53.14,51.34
1,334212,Black,NaN,NC,South,South Atlantic,Malignant neoplasm of unspecified site of left...,C773,60,10875.30,...,48.63,58.14,77.26,80.05,82.88,82.09,78.85,64.60,50.57,48.10
2,571362,NaN,COMMERCIAL,TX,South,West South Central,Malignant neoplasm of upper-inner quadrant of ...,C773,54,18716.73,...,57.82,59.95,77.79,82.45,82.44,80.77,72.16,59.31,48.25,42.13
3,907331,NaN,COMMERCIAL,TN,South,East South Central,Malignant neoplasm of other specified sites of...,C7951,63,7804.79,...,47.57,53.50,71.31,75.20,76.96,75.78,74.87,61.06,44.31,42.83
4,208382,Asian,NaN,WA,West,Pacific,Malig neoplm of upper-outer quadrant of right ...,C787,62,28628.29,...,41.02,46.25,56.92,57.88,66.16,65.21,57.52,49.53,43.75,38.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,649011,White,NaN,CO,West,Mountain,Malignant neoplasm of unsp site of right femal...,C773,63,33840.19,...,43.24,48.85,61.47,72.59,74.94,72.44,67.41,49.12,38.81,33.29
5642,869024,NaN,COMMERCIAL,GA,South,South Atlantic,Malig neoplasm of upper-inner quadrant of left...,C7989,57,40162.73,...,50.20,56.76,72.53,77.56,78.82,77.39,78.71,64.14,47.25,44.86
5643,304800,Asian,COMMERCIAL,CA,West,Pacific,"Malignant neoplasm of breast (female), unspeci...",C7951,44,39504.88,...,59.25,62.98,63.26,67.57,76.04,75.15,71.05,68.75,65.27,59.42
5644,267406,White,MEDICARE ADVANTAGE,CA,West,Pacific,Malignant neoplasm of unsp site of right femal...,C779,66,40982.11,...,60.26,64.84,64.40,69.37,78.29,77.42,73.15,70.26,65.92,59.39


In [14]:
final_test_data['patient_race']=final_test_data['patient_race'].fillna("Race Not Available")

In [15]:
final_test_data

,patient_id,patient_race,payer_type,patient_state,Region,Division,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,patient_age,population,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
0,730681,Race Not Available,COMMERCIAL,LA,South,West South Central,Malignant neoplasm of axillary tail of female ...,C7981,55,4639.07,...,62.21,62.23,78.34,81.96,83.58,82.22,80.20,69.73,53.14,51.34
1,334212,Black,NaN,NC,South,South Atlantic,Malignant neoplasm of unspecified site of left...,C773,60,10875.30,...,48.63,58.14,77.26,80.05,82.88,82.09,78.85,64.60,50.57,48.10
2,571362,Race Not Available,COMMERCIAL,TX,South,West South Central,Malignant neoplasm of upper-inner quadrant of ...,C773,54,18716.73,...,57.82,59.95,77.79,82.45,82.44,80.77,72.16,59.31,48.25,42.13
3,907331,Race Not Available,COMMERCIAL,TN,South,East South Central,Malignant neoplasm of other specified sites of...,C7951,63,7804.79,...,47.57,53.50,71.31,75.20,76.96,75.78,74.87,61.06,44.31,42.83
4,208382,Asian,NaN,WA,West,Pacific,Malig neoplm of upper-outer quadrant of right ...,C787,62,28628.29,...,41.02,46.25,56.92,57.88,66.16,65.21,57.52,49.53,43.75,38.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,649011,White,NaN,CO,West,Mountain,Malignant neoplasm of unsp site of right femal...,C773,63,33840.19,...,43.24,48.85,61.47,72.59,74.94,72.44,67.41,49.12,38.81,33.29
5642,869024,Race Not Available,COMMERCIAL,GA,South,South Atlantic,Malig neoplasm of upper-inner quadrant of left...,C7989,57,40162.73,...,50.20,56.76,72.53,77.56,78.82,77.39,78.71,64.14,47.25,44.86
5643,304800,Asian,COMMERCIAL,CA,West,Pacific,"Malignant neoplasm of breast (female), unspeci...",C7951,44,39504.88,...,59.25,62.98,63.26,67.57,76.04,75.15,71.05,68.75,65.27,59.42
5644,267406,White,MEDICARE ADVANTAGE,CA,West,Pacific,Malignant neoplasm of unsp site of right femal...,C779,66,40982.11,...,60.26,64.84,64.40,69.37,78.29,77.42,73.15,70.26,65.92,59.39


In [16]:
final_test_data['payer_type']=final_test_data['payer_type'].fillna("Payer Type Not Available")

In [17]:
final_test_data

,patient_id,patient_race,payer_type,patient_state,Region,Division,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,patient_age,population,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
0,730681,Race Not Available,COMMERCIAL,LA,South,West South Central,Malignant neoplasm of axillary tail of female ...,C7981,55,4639.07,...,62.21,62.23,78.34,81.96,83.58,82.22,80.20,69.73,53.14,51.34
1,334212,Black,Payer Type Not Available,NC,South,South Atlantic,Malignant neoplasm of unspecified site of left...,C773,60,10875.30,...,48.63,58.14,77.26,80.05,82.88,82.09,78.85,64.60,50.57,48.10
2,571362,Race Not Available,COMMERCIAL,TX,South,West South Central,Malignant neoplasm of upper-inner quadrant of ...,C773,54,18716.73,...,57.82,59.95,77.79,82.45,82.44,80.77,72.16,59.31,48.25,42.13
3,907331,Race Not Available,COMMERCIAL,TN,South,East South Central,Malignant neoplasm of other specified sites of...,C7951,63,7804.79,...,47.57,53.50,71.31,75.20,76.96,75.78,74.87,61.06,44.31,42.83
4,208382,Asian,Payer Type Not Available,WA,West,Pacific,Malig neoplm of upper-outer quadrant of right ...,C787,62,28628.29,...,41.02,46.25,56.92,57.88,66.16,65.21,57.52,49.53,43.75,38.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,649011,White,Payer Type Not Available,CO,West,Mountain,Malignant neoplasm of unsp site of right femal...,C773,63,33840.19,...,43.24,48.85,61.47,72.59,74.94,72.44,67.41,49.12,38.81,33.29
5642,869024,Race Not Available,COMMERCIAL,GA,South,South Atlantic,Malig neoplasm of upper-inner quadrant of left...,C7989,57,40162.73,...,50.20,56.76,72.53,77.56,78.82,77.39,78.71,64.14,47.25,44.86
5643,304800,Asian,COMMERCIAL,CA,West,Pacific,"Malignant neoplasm of breast (female), unspeci...",C7951,44,39504.88,...,59.25,62.98,63.26,67.57,76.04,75.15,71.05,68.75,65.27,59.42
5644,267406,White,MEDICARE ADVANTAGE,CA,West,Pacific,Malignant neoplasm of unsp site of right femal...,C779,66,40982.11,...,60.26,64.84,64.40,69.37,78.29,77.42,73.15,70.26,65.92,59.39


In [18]:
final_test_data['patient_state'].isna().sum()

0

In [19]:
final_test_data['Region'].isna().sum()

0

In [20]:
final_test_data['Division'].isna().sum()

0

In [21]:
final_test_data['breast_cancer_diagnosis_desc'].isna().sum()

0

In [22]:
final_test_data['metastatic_cancer_diagnosis_code'].isna().sum()

0

In [23]:
#final_column_list[8:]

In [24]:
for i in final_column_list[8:]:
    final_test_data[i]=final_test_data[i].fillna(final_test_data[i].mean())

In [25]:
final_test_data

,patient_id,patient_race,payer_type,patient_state,Region,Division,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,patient_age,population,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
0,730681,Race Not Available,COMMERCIAL,LA,South,West South Central,Malignant neoplasm of axillary tail of female ...,C7981,55,4639.07,...,62.21,62.23,78.34,81.96,83.58,82.22,80.20,69.73,53.14,51.34
1,334212,Black,Payer Type Not Available,NC,South,South Atlantic,Malignant neoplasm of unspecified site of left...,C773,60,10875.30,...,48.63,58.14,77.26,80.05,82.88,82.09,78.85,64.60,50.57,48.10
2,571362,Race Not Available,COMMERCIAL,TX,South,West South Central,Malignant neoplasm of upper-inner quadrant of ...,C773,54,18716.73,...,57.82,59.95,77.79,82.45,82.44,80.77,72.16,59.31,48.25,42.13
3,907331,Race Not Available,COMMERCIAL,TN,South,East South Central,Malignant neoplasm of other specified sites of...,C7951,63,7804.79,...,47.57,53.50,71.31,75.20,76.96,75.78,74.87,61.06,44.31,42.83
4,208382,Asian,Payer Type Not Available,WA,West,Pacific,Malig neoplm of upper-outer quadrant of right ...,C787,62,28628.29,...,41.02,46.25,56.92,57.88,66.16,65.21,57.52,49.53,43.75,38.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,649011,White,Payer Type Not Available,CO,West,Mountain,Malignant neoplasm of unsp site of right femal...,C773,63,33840.19,...,43.24,48.85,61.47,72.59,74.94,72.44,67.41,49.12,38.81,33.29
5642,869024,Race Not Available,COMMERCIAL,GA,South,South Atlantic,Malig neoplasm of upper-inner quadrant of left...,C7989,57,40162.73,...,50.20,56.76,72.53,77.56,78.82,77.39,78.71,64.14,47.25,44.86
5643,304800,Asian,COMMERCIAL,CA,West,Pacific,"Malignant neoplasm of breast (female), unspeci...",C7951,44,39504.88,...,59.25,62.98,63.26,67.57,76.04,75.15,71.05,68.75,65.27,59.42
5644,267406,White,MEDICARE ADVANTAGE,CA,West,Pacific,Malignant neoplasm of unsp site of right femal...,C779,66,40982.11,...,60.26,64.84,64.40,69.37,78.29,77.42,73.15,70.26,65.92,59.39


In [26]:
for i in cont_columns:
    final_test_data[i] = final_test_data[i].astype(float)

In [27]:
final_test_data

,patient_id,patient_race,payer_type,patient_state,Region,Division,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,patient_age,population,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
0,730681,Race Not Available,COMMERCIAL,LA,South,West South Central,Malignant neoplasm of axillary tail of female ...,C7981,55.0,4639.07,...,62.21,62.23,78.34,81.96,83.58,82.22,80.20,69.73,53.14,51.34
1,334212,Black,Payer Type Not Available,NC,South,South Atlantic,Malignant neoplasm of unspecified site of left...,C773,60.0,10875.30,...,48.63,58.14,77.26,80.05,82.88,82.09,78.85,64.60,50.57,48.10
2,571362,Race Not Available,COMMERCIAL,TX,South,West South Central,Malignant neoplasm of upper-inner quadrant of ...,C773,54.0,18716.73,...,57.82,59.95,77.79,82.45,82.44,80.77,72.16,59.31,48.25,42.13
3,907331,Race Not Available,COMMERCIAL,TN,South,East South Central,Malignant neoplasm of other specified sites of...,C7951,63.0,7804.79,...,47.57,53.50,71.31,75.20,76.96,75.78,74.87,61.06,44.31,42.83
4,208382,Asian,Payer Type Not Available,WA,West,Pacific,Malig neoplm of upper-outer quadrant of right ...,C787,62.0,28628.29,...,41.02,46.25,56.92,57.88,66.16,65.21,57.52,49.53,43.75,38.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,649011,White,Payer Type Not Available,CO,West,Mountain,Malignant neoplasm of unsp site of right femal...,C773,63.0,33840.19,...,43.24,48.85,61.47,72.59,74.94,72.44,67.41,49.12,38.81,33.29
5642,869024,Race Not Available,COMMERCIAL,GA,South,South Atlantic,Malig neoplasm of upper-inner quadrant of left...,C7989,57.0,40162.73,...,50.20,56.76,72.53,77.56,78.82,77.39,78.71,64.14,47.25,44.86
5643,304800,Asian,COMMERCIAL,CA,West,Pacific,"Malignant neoplasm of breast (female), unspeci...",C7951,44.0,39504.88,...,59.25,62.98,63.26,67.57,76.04,75.15,71.05,68.75,65.27,59.42
5644,267406,White,MEDICARE ADVANTAGE,CA,West,Pacific,Malignant neoplasm of unsp site of right femal...,C779,66.0,40982.11,...,60.26,64.84,64.40,69.37,78.29,77.42,73.15,70.26,65.92,59.39


In [28]:
x_cat_test=final_test_data[cat_columns]

In [29]:
x_cat_test

,patient_race,payer_type,patient_state,Region,Division,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code
0,Race Not Available,COMMERCIAL,LA,South,West South Central,Malignant neoplasm of axillary tail of female ...,C7981
1,Black,Payer Type Not Available,NC,South,South Atlantic,Malignant neoplasm of unspecified site of left...,C773
2,Race Not Available,COMMERCIAL,TX,South,West South Central,Malignant neoplasm of upper-inner quadrant of ...,C773
3,Race Not Available,COMMERCIAL,TN,South,East South Central,Malignant neoplasm of other specified sites of...,C7951
4,Asian,Payer Type Not Available,WA,West,Pacific,Malig neoplm of upper-outer quadrant of right ...,C787
...,...,...,...,...,...,...,...
5641,White,Payer Type Not Available,CO,West,Mountain,Malignant neoplasm of unsp site of right femal...,C773
5642,Race Not Available,COMMERCIAL,GA,South,South Atlantic,Malig neoplasm of upper-inner quadrant of left...,C7989
5643,Asian,COMMERCIAL,CA,West,Pacific,"Malignant neoplasm of breast (female), unspeci...",C7951
5644,White,MEDICARE ADVANTAGE,CA,West,Pacific,Malignant neoplasm of unsp site of right femal...,C779


In [30]:
x_cont_test=final_test_data[cont_columns]

In [31]:
x_cont_test

,patient_age,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
0,55.0,4639.07,72.66,41.50,11.40,13.44,11.42,11.45,12.56,13.08,...,62.21,62.23,78.34,81.96,83.58,82.22,80.20,69.73,53.14,51.34
1,60.0,10875.30,217.91,39.64,11.23,13.72,15.01,12.02,11.58,11.77,...,48.63,58.14,77.26,80.05,82.88,82.09,78.85,64.60,50.57,48.10
2,54.0,18716.73,1019.51,30.37,11.00,18.86,23.11,12.24,9.88,9.12,...,57.82,59.95,77.79,82.45,82.44,80.77,72.16,59.31,48.25,42.13
3,63.0,7804.79,140.05,44.32,10.19,12.66,11.70,10.53,12.13,14.91,...,47.57,53.50,71.31,75.20,76.96,75.78,74.87,61.06,44.31,42.83
4,62.0,28628.29,1091.83,39.68,12.14,12.46,11.32,15.21,14.45,14.11,...,41.02,46.25,56.92,57.88,66.16,65.21,57.52,49.53,43.75,38.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,63.0,33840.19,1214.02,38.10,12.79,12.24,13.20,14.93,14.41,12.91,...,43.24,48.85,61.47,72.59,74.94,72.44,67.41,49.12,38.81,33.29
5642,57.0,40162.73,875.12,37.73,13.42,14.39,12.03,13.81,14.03,14.04,...,50.20,56.76,72.53,77.56,78.82,77.39,78.71,64.14,47.25,44.86
5643,44.0,39504.88,4008.49,37.55,11.49,11.44,14.51,16.61,14.29,13.65,...,59.25,62.98,63.26,67.57,76.04,75.15,71.05,68.75,65.27,59.42
5644,66.0,40982.11,3008.77,37.06,10.83,15.64,16.01,13.03,12.58,12.30,...,60.26,64.84,64.40,69.37,78.29,77.42,73.15,70.26,65.92,59.39


In [32]:
# Load StandardScaler
with open('scaler.pkl', 'rb') as f:
    loaded_scaler = pickle.load(f)

# Load OneHotEncoder
with open('encoder.pkl', 'rb') as f:
    loaded_encoder = pickle.load(f)

# Load RandomForestClassifier
with open('model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)


In [33]:
x_cat_test_new=pd.DataFrame(loaded_encoder.transform(x_cat_test),columns=list(loaded_encoder.get_feature_names_out()))

In [34]:
x_cat_test_new

,patient_race_Asian,patient_race_Black,patient_race_Hispanic,patient_race_Other,patient_race_Race Not Available,patient_race_White,payer_type_COMMERCIAL,payer_type_MEDICAID,payer_type_MEDICARE ADVANTAGE,payer_type_Payer Type Not Available,...,metastatic_cancer_diagnosis_code_C7952,metastatic_cancer_diagnosis_code_C7960,metastatic_cancer_diagnosis_code_C7961,metastatic_cancer_diagnosis_code_C7970,metastatic_cancer_diagnosis_code_C7971,metastatic_cancer_diagnosis_code_C7972,metastatic_cancer_diagnosis_code_C7981,metastatic_cancer_diagnosis_code_C7982,metastatic_cancer_diagnosis_code_C7989,metastatic_cancer_diagnosis_code_C799
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5642,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5643,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5644,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [35]:
x_cont_test_new=pd.DataFrame(loaded_scaler.transform(x_cont_test),columns=cont_columns)

In [36]:
x_cont_test_new

,patient_age,population,density,age_median,age_under_10,age_10_to_19,age_20s,age_30s,age_40s,age_50s,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
0,-0.348553,-1.255469,-0.701950,0.196723,0.218328,0.374461,-0.559971,-0.604412,0.405601,-0.409395,...,1.467180,1.023895,1.747225,1.467847,1.412472,1.331134,1.483411,1.222311,0.692642,0.938119
1,0.032222,-0.705128,-0.594599,-0.316400,0.084481,0.557987,0.740915,-0.351359,-0.507455,-1.310172,...,0.314796,0.648239,1.567715,1.162487,1.270773,1.304162,1.265606,0.663722,0.480177,0.642335
2,-0.424709,-0.013129,-0.002157,-2.873739,-0.096605,3.926985,3.676063,-0.253689,-2.091329,-3.132354,...,1.094650,0.814483,1.655808,1.546186,1.181705,1.030300,0.186263,0.087711,0.288380,0.097325
3,0.260688,-0.976098,-0.652144,0.974683,-0.734343,-0.136787,-0.458509,-1.012849,0.004974,0.848941,...,0.224846,0.222068,0.578750,0.387095,0.072402,-0.004983,0.623486,0.278262,-0.037344,0.161229
4,0.184533,0.861557,0.051293,-0.305365,0.800953,-0.267877,-0.596207,1.064852,2.166496,0.298849,...,-0.330980,-0.443825,-1.813051,-2.381932,-2.113815,-2.197956,-2.175708,-0.977202,-0.083640,-0.249583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,0.260688,1.321502,0.141600,-0.741244,1.312718,-0.412075,0.085037,0.940545,2.129229,-0.526290,...,-0.142593,-0.205022,-1.056783,-0.030177,-0.336501,-0.697937,-0.580087,-1.021846,-0.492035,-0.709692
5642,-0.196243,1.879460,-0.108872,-0.843317,1.808737,0.997136,-0.338929,0.443318,1.775186,0.250716,...,0.448025,0.521490,0.781529,0.764399,0.448917,0.329046,1.243019,0.613634,0.205709,0.346550
5643,-1.186261,1.821405,2.206922,-0.892974,0.289187,-0.936433,0.559734,1.686387,2.017426,-0.017454,...,1.215998,1.092780,-0.759263,-0.832747,-0.113831,-0.135690,0.007179,1.115602,1.695442,1.675754
5644,0.489154,1.951770,1.468055,-1.028151,-0.230451,1.816445,1.103279,0.097034,0.424235,-0.945736,...,1.301705,1.263617,-0.569780,-0.544973,0.341631,0.335271,0.345986,1.280021,1.749179,1.673015


In [37]:
X_test=pd.concat((x_cat_test_new,x_cont_test_new),axis=1)

In [38]:
X_test

,patient_race_Asian,patient_race_Black,patient_race_Hispanic,patient_race_Other,patient_race_Race Not Available,patient_race_White,payer_type_COMMERCIAL,payer_type_MEDICAID,payer_type_MEDICARE ADVANTAGE,payer_type_Payer Type Not Available,...,Average of Mar-18,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18
0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.467180,1.023895,1.747225,1.467847,1.412472,1.331134,1.483411,1.222311,0.692642,0.938119
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.314796,0.648239,1.567715,1.162487,1.270773,1.304162,1.265606,0.663722,0.480177,0.642335
2,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,1.094650,0.814483,1.655808,1.546186,1.181705,1.030300,0.186263,0.087711,0.288380,0.097325
3,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.224846,0.222068,0.578750,0.387095,0.072402,-0.004983,0.623486,0.278262,-0.037344,0.161229
4,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,-0.330980,-0.443825,-1.813051,-2.381932,-2.113815,-2.197956,-2.175708,-0.977202,-0.083640,-0.249583
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,-0.142593,-0.205022,-1.056783,-0.030177,-0.336501,-0.697937,-0.580087,-1.021846,-0.492035,-0.709692
5642,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.448025,0.521490,0.781529,0.764399,0.448917,0.329046,1.243019,0.613634,0.205709,0.346550
5643,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.215998,1.092780,-0.759263,-0.832747,-0.113831,-0.135690,0.007179,1.115602,1.695442,1.675754
5644,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,...,1.301705,1.263617,-0.569780,-0.544973,0.341631,0.335271,0.345986,1.280021,1.749179,1.673015


In [39]:
# Load RandomForestClassifier
with open('rf_model.pkl', 'rb') as f:
    rf_model = pickle.load(f)

In [40]:
# Load RandomForestClassifier
with open('fs_model.pkl', 'rb') as f:
    fs_model = pickle.load(f)

In [41]:
X_test_new=fs_model.transform(X_test)

In [42]:
X_test_new.shape

(5646, 25)

In [43]:
y_pred_rf=rf_model.predict(X_test_new)

In [44]:
y_pred_rf

array([0, 0, 1, ..., 1, 0, 0])

In [45]:
Counter(y_pred_rf)

Counter({0: 4371, 1: 1275})

In [46]:
final_prediction=['Within 3 months' if x==0 else 'Greaterthan 3 months' for x in y_pred_rf]

In [47]:
Counter(final_prediction)

Counter({'Within 3 months': 4371, 'Greaterthan 3 months': 1275})

In [123]:
#final_prediction

In [193]:
final_test_data['metastatic_diagnosis_period_bucket']=final_prediction

In [194]:
final_test_data

,patient_id,patient_race,payer_type,patient_state,Region,Division,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,patient_age,population,...,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18,metastatic_diagnosis_period_bucket
0,730681,Race Not Available,COMMERCIAL,LA,South,West South Central,Malignant neoplasm of axillary tail of female ...,C7981,55.0,4639.07,...,62.23,78.34,81.96,83.58,82.22,80.20,69.73,53.14,51.34,Within 3 months
1,334212,Black,Payer Type Not Available,NC,South,South Atlantic,Malignant neoplasm of unspecified site of left...,C773,60.0,10875.30,...,58.14,77.26,80.05,82.88,82.09,78.85,64.60,50.57,48.10,Within 3 months
2,571362,Race Not Available,COMMERCIAL,TX,South,West South Central,Malignant neoplasm of upper-inner quadrant of ...,C773,54.0,18716.73,...,59.95,77.79,82.45,82.44,80.77,72.16,59.31,48.25,42.13,Greaterthan 3 months
3,907331,Race Not Available,COMMERCIAL,TN,South,East South Central,Malignant neoplasm of other specified sites of...,C7951,63.0,7804.79,...,53.50,71.31,75.20,76.96,75.78,74.87,61.06,44.31,42.83,Greaterthan 3 months
4,208382,Asian,Payer Type Not Available,WA,West,Pacific,Malig neoplm of upper-outer quadrant of right ...,C787,62.0,28628.29,...,46.25,56.92,57.88,66.16,65.21,57.52,49.53,43.75,38.33,Within 3 months
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,649011,White,Payer Type Not Available,CO,West,Mountain,Malignant neoplasm of unsp site of right femal...,C773,63.0,33840.19,...,48.85,61.47,72.59,74.94,72.44,67.41,49.12,38.81,33.29,Within 3 months
5642,869024,Race Not Available,COMMERCIAL,GA,South,South Atlantic,Malig neoplasm of upper-inner quadrant of left...,C7989,57.0,40162.73,...,56.76,72.53,77.56,78.82,77.39,78.71,64.14,47.25,44.86,Within 3 months
5643,304800,Asian,COMMERCIAL,CA,West,Pacific,"Malignant neoplasm of breast (female), unspeci...",C7951,44.0,39504.88,...,62.98,63.26,67.57,76.04,75.15,71.05,68.75,65.27,59.42,Greaterthan 3 months
5644,267406,White,MEDICARE ADVANTAGE,CA,West,Pacific,Malignant neoplasm of unsp site of right femal...,C779,66.0,40982.11,...,64.84,64.40,69.37,78.29,77.42,73.15,70.26,65.92,59.39,Within 3 months


In [195]:
final_result=final_test_data[['patient_id','metastatic_diagnosis_period_bucket']]

In [196]:
final_result.to_csv('result.csv',index=False)

In [48]:
#svm

In [50]:
# Load RandomForestClassifier
with open('svm_model.pkl', 'rb') as f:
    svm_model = pickle.load(f)

In [52]:
X_test_new.shape

(5646, 25)

In [53]:
y_pred_svm=svm_model.predict(X_test_new)

In [54]:
Counter(y_pred_svm)

Counter({0: 4371, 1: 1275})

In [55]:
final_prediction_svm=['Within 3 months' if x==0 else 'Greaterthan 3 months' for x in y_pred_svm]

In [56]:
Counter(final_prediction_svm)

Counter({'Within 3 months': 4371, 'Greaterthan 3 months': 1275})

In [57]:
final_test_data['metastatic_diagnosis_period_bucket_svm']=final_prediction_svm

In [58]:
final_test_data

,patient_id,patient_race,payer_type,patient_state,Region,Division,breast_cancer_diagnosis_desc,metastatic_cancer_diagnosis_code,patient_age,population,...,Average of Apr-18,Average of May-18,Average of Jun-18,Average of Jul-18,Average of Aug-18,Average of Sep-18,Average of Oct-18,Average of Nov-18,Average of Dec-18,metastatic_diagnosis_period_bucket_svm
0,730681,Race Not Available,COMMERCIAL,LA,South,West South Central,Malignant neoplasm of axillary tail of female ...,C7981,55.0,4639.07,...,62.23,78.34,81.96,83.58,82.22,80.20,69.73,53.14,51.34,Within 3 months
1,334212,Black,Payer Type Not Available,NC,South,South Atlantic,Malignant neoplasm of unspecified site of left...,C773,60.0,10875.30,...,58.14,77.26,80.05,82.88,82.09,78.85,64.60,50.57,48.10,Within 3 months
2,571362,Race Not Available,COMMERCIAL,TX,South,West South Central,Malignant neoplasm of upper-inner quadrant of ...,C773,54.0,18716.73,...,59.95,77.79,82.45,82.44,80.77,72.16,59.31,48.25,42.13,Greaterthan 3 months
3,907331,Race Not Available,COMMERCIAL,TN,South,East South Central,Malignant neoplasm of other specified sites of...,C7951,63.0,7804.79,...,53.50,71.31,75.20,76.96,75.78,74.87,61.06,44.31,42.83,Greaterthan 3 months
4,208382,Asian,Payer Type Not Available,WA,West,Pacific,Malig neoplm of upper-outer quadrant of right ...,C787,62.0,28628.29,...,46.25,56.92,57.88,66.16,65.21,57.52,49.53,43.75,38.33,Within 3 months
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5641,649011,White,Payer Type Not Available,CO,West,Mountain,Malignant neoplasm of unsp site of right femal...,C773,63.0,33840.19,...,48.85,61.47,72.59,74.94,72.44,67.41,49.12,38.81,33.29,Within 3 months
5642,869024,Race Not Available,COMMERCIAL,GA,South,South Atlantic,Malig neoplasm of upper-inner quadrant of left...,C7989,57.0,40162.73,...,56.76,72.53,77.56,78.82,77.39,78.71,64.14,47.25,44.86,Within 3 months
5643,304800,Asian,COMMERCIAL,CA,West,Pacific,"Malignant neoplasm of breast (female), unspeci...",C7951,44.0,39504.88,...,62.98,63.26,67.57,76.04,75.15,71.05,68.75,65.27,59.42,Greaterthan 3 months
5644,267406,White,MEDICARE ADVANTAGE,CA,West,Pacific,Malignant neoplasm of unsp site of right femal...,C779,66.0,40982.11,...,64.84,64.40,69.37,78.29,77.42,73.15,70.26,65.92,59.39,Within 3 months


In [59]:
final_result_svm=final_test_data[['patient_id','metastatic_diagnosis_period_bucket_svm']]

In [60]:
final_result_svm

,patient_id,metastatic_diagnosis_period_bucket_svm
0,730681,Within 3 months
1,334212,Within 3 months
2,571362,Greaterthan 3 months
3,907331,Greaterthan 3 months
4,208382,Within 3 months
...,...,...
5641,649011,Within 3 months
5642,869024,Within 3 months
5643,304800,Greaterthan 3 months
5644,267406,Within 3 months


In [61]:
final_result_svm.to_csv('result_svm.csv', index=False)